In [1]:
import numpy as np, pandas as pd
import json
import ast 
from textblob import TextBlob
import nltk
import torch
import pickle
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [31]:
# !conda update pandas --y

In [4]:
# comment out to avoid overwriting original data
# load train data
train = pd.read_csv("data/train.csv")

In [5]:
train.shape

(1314, 4)

### Loading Embedding dictionary

In [45]:
with open("data/dict_embeddings3.pickle", "rb") as f:
    d1 = pickle.load(f)

with open("data/dict_embeddings4.pickle", "rb") as f:
    d2 = pickle.load(f)

In [46]:
# key - sentence / question , val - vector
dict_emb = dict(d1)
dict_emb.update(d2)

In [47]:
dict_emb['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'][0]

array([ 0.00746889,  0.02421027,  0.07833809, ...,  0.04228063,
        0.06608193, -0.00488943], dtype=float32)

In [49]:
len(dict_emb)

4823

In [50]:
del d1, d2

## Data Processing

In [51]:
# find the index of the sentence that contains answer
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [52]:
train.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,euclidean_dis2,minkowski_dis,chebyshev_dis,pred_idx_cos,pred_idx_euc,pred_idx_min,pred_idx_che
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[-0.00991958, -0.035521798, 0.06675484, -0.02...","[[-0.0058745374, -0.07653264, 0.08423636, -0.0...","[0.25390303134918213, 0.22334301471710205, 0.2...","[1.7075382, 1.7290226, 1.8517298, 1.4138652, 1...","[1.3067281246185303, 1.3149230480194092, 1.360...","[60.20494, 61.16893, 64.51268, 56.288124, 57.5...","[0.1423744, 0.10892366, 0.14755285, 0.09070109...",5,5,5,3
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[-0.00991958, -0.035521798, 0.06675484, -0.02...","[[0.017040426, -0.06193916, 0.07699205, -0.010...","[0.2634468674659729, 0.2264377474784851, 0.219...","[2.1715844, 2.0053604, 1.9923788, 1.6267016, 2...","[1.4736297130584717, 1.416107416152954, 1.4115...","[68.16332, 66.02714, 65.82747, 58.949272, 67.1...","[0.11970146, 0.118082315, 0.12687306, 0.134611...",3,3,3,1
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[-0.00991958, -0.035521798, 0.06675484, -0.02...","[[0.013891986, -0.051962323, 0.069856785, -0.0...","[0.24388277530670166, 0.2215394377708435, 0.22...","[2.072908, 1.9996178, 2.110013, 1.616806, 1.98...","[1.4397597312927246, 1.4140784740447998, 1.452...","[66.24587, 65.72272, 67.814575, 58.87599, 64.8...","[0.12093631, 0.12933947, 0.13097605, 0.1125107...",3,3,3,4


In [53]:
train.shape

(1314, 17)

In [54]:
train.dropna(inplace=True)

In [55]:
train.shape

(1314, 17)

In [56]:
dict_emb['Architecturally, the school has a Catholic character.']

array([[ 0.00746889, -0.05086312,  0.00736477, ...,  0.04118384,
         0.01421824, -0.0138564 ]], dtype=float32)

In [57]:
def process_data(train):
    
    print("step 1")
    train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
    print("step 2")
    train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train   

In [58]:
train = process_data(train)

step 1
step 2
step 3
step 4


In [59]:
train.head(3)

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,euclidean_dis2,minkowski_dis,chebyshev_dis,pred_idx_cos,pred_idx_euc,pred_idx_min,pred_idx_che
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.007468889, 0.024210269, 0.07833809, -0.017...","[0.25390303134918213, 0.22334301471710205, 0.2...","[1.7075382, 1.7290226, 1.8517298, 1.4138652, 1...","[1.3067281246185303, 1.3149230480194092, 1.360...","[60.20494, 61.16893, 64.51268, 56.288124, 57.5...","[0.1423744, 0.10892366, 0.14755285, 0.09070109...",5,5,5,3
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.10951651, 0.11030624, 0.05210008, 0.030539...","[0.2634468674659729, 0.2264377474784851, 0.219...","[2.1715844, 2.0053604, 1.9923788, 1.6267016, 2...","[1.4736297130584717, 1.416107416152954, 1.4115...","[68.16332, 66.02714, 65.82747, 58.949272, 67.1...","[0.11970146, 0.118082315, 0.12687306, 0.134611...",3,3,3,1
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.007468889, -0.028770322, 0.13643634, -0.00...","[0.24388277530670166, 0.2215394377708435, 0.22...","[2.072908, 1.9996178, 2.110013, 1.616806, 1.98...","[1.4397597312927246, 1.4140784740447998, 1.452...","[66.24587, 65.72272, 67.814575, 58.87599, 64.8...","[0.12093631, 0.12933947, 0.13097605, 0.1125107...",3,3,3,4


In [60]:
train.iloc[0,:]['sent_emb']

[array([ 0.00746889, -0.05086312,  0.00736477, ...,  0.04118384,
         0.01421824, -0.0138564 ], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.00746889, 0.00995224, 0.19095322, ..., 0.00119954, 0.04134529,
        0.03366186], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([ 0.00746889, -0.05682991,  0.11969507, ..., -0.01752904,
         0.01482137, -0.00012652], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([ 0.00746889, -0.01367913,  0.1417282 , ...,  0.0098886 ,
        -0.00070966,  0.06526422], dtype=float32)]

## Predicted Cosine & Euclidean & other Index

In [61]:
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li   

In [62]:
def euc_dist(x):
    li = []
    for item in x["sent_emb"]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.euclidean(item,x["quest_emb"][0]))
    return li   

In [63]:
# actually manhatten distance as p = 1
def min_dist(x,p=1):
    li = []
    for item in x["sent_emb"]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.minkowski(item,x["quest_emb"][0],p))
    return li  

In [64]:
def che_dist(x):
    li = []
    for item in x["sent_emb"]:
#         print(item)
#         print(x["quest_emb"][0])
#         print()
        li.append(spatial.distance.chebyshev(item,x["quest_emb"][0]))
    return li  

In [65]:
def pred_idx(distances):
    return np.argmin(distances)   

In [66]:
def predictions(train):
    
    train["cosine_sim"] = train.apply(cosine_sim, axis = 1)
    train["diff"] = (train["quest_emb"] - train["sent_emb"])**2
    train["euclidean_dis"] = train["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    train["euclidean_dis2"]= train.apply(euc_dist,axis = 1)
    train["minkowski_dis"] = train.apply(min_dist,axis = 1)
    train["chebyshev_dis"] = train.apply(che_dist,axis = 1)
    del train["diff"]
    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim"].apply(lambda x: pred_idx(x))
    train["pred_idx_euc"] = train["euclidean_dis"].apply(lambda x: pred_idx(x))
    train["pred_idx_min"] = train["minkowski_dis"].apply(lambda x: pred_idx(x))
    train["pred_idx_che"] = train["chebyshev_dis"].apply(lambda x: pred_idx(x))



    return train
    

In [67]:
predicted = predictions(train)

cosine start


In [85]:
predicted

,context,question,answer_start,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,euclidean_dis2,minkowski_dis,chebyshev_dis,pred_idx_cos,pred_idx_euc,pred_idx_min,pred_idx_che
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...",5,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.007468889, 0.024210269, 0.07833809, -0.017...","[0.6866069734096527, nan, 0.5840356051921844, ...","[7.33839222452402, 6.306084297834602, 7.334488...","[2.708946704864502, 2.511191808252528, 2.70822...","[115.95091, 107.00983111224195, 117.443405, 10...","[0.40420753, 0.3282351493835449, 0.29667524, 0...",1,1,1,2
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...",2,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.10951651, 0.11030624, 0.05210008, 0.030539...","[0.9777406323701143, nan, 0.7699109315872192, ...","[19.410055392295288, 15.479153964177083, 17.20...","[4.405684471130371, 3.9343555970675914, 4.1478...","[209.98334, 182.10363173291626, 199.47734, 182...","[0.3185168, 0.31212911009788513, 0.30937737, 0...",1,1,1,2
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,"[Architecturally, the school has a Catholic ch...",3,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.007468889, -0.028770322, 0.13643634, -0.00...","[0.5929965972900391, nan, 0.5739337205886841, ...","[5.201025166964836, 4.475611479051657, 6.22011...","[2.2805755138397217, 2.1155641042170426, 2.494...","[97.44212, 94.82440084673362, 107.91632, 94.82...","[0.31453472, 0.23689861595630646, 0.27307037, ...",1,4,4,4
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...",4,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.0711433, 0.054118328, -0.013959831, 0.0531...","[0.9819837901741266, nan, 0.8030855655670166, ...","[18.38671648690265, 14.375967946842614, 16.894...","[4.287973403930664, 3.7915653689264825, 4.1102...","[202.63652, 172.70786534426588, 198.25499, 172...","[0.32432848, 0.3129982650279999, 0.31024653, 0...",1,1,1,2
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...",1,"[[0.007468889, -0.050863117, 0.0073647713, -0....","[[0.007468889, -0.021445127, 0.069521695, -0.0...","[0.7048581540584564, nan, 0.5938917696475983, ...","[6.421781746472394, 4.810748909596436, 6.60844...","[2.534123420715332, 2.193341949992394, 2.57068...","[110.248955, 99.35428729441992, 107.84592, 99....","[0.3375644, 0.260966032743454, 0.24375708, 0.2...",1,1,1,4
5,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,248,September 1876,"[As at most other universities, Notre Dame's s...",2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.016918726, 0.120841, -0.006960277, 0.05206...","[nan, 0.38319146633148193, nan, 0.320999383926...","[15.49875419443368, 16.473732412377696, 15.498...","[3.936845716361473, 4.058784484863281, 3.93684...","[185.78077896612194, 193.62622, 185.7807789661...","[0.2484964281320572, 0.2881556, 0.248496428132...",0,3,3,0
6,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,441,twice,"[As at most other universities, Notre Dame's s...",3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.007468889, -0.0335015, 0.11457597, -0.0090...","[nan, 0.9407149367034435, nan, 0.9315105974674...","[5.003687629388903, 29.16323049176077, 5.00368...","[2.2368924045176835, 5.400299072265625, 2.2368...","[99.20761936692816, 268.18872, 9

## Accuracy

In [69]:
def accuracy(target, predicted):
    
    acc = (target==predicted).sum()/len(target)
    
    return acc

### Accuracy for  euclidean Distance

In [84]:
print(accuracy(predicted["target"], predicted["pred_idx_euc"]))

0.2929984779299848


### Accuracy for Cosine Similarity and others

In [83]:
print(accuracy(predicted["target"], predicted["pred_idx_cos"]))

0.24885844748858446


In [72]:
print(accuracy(predicted["target"], predicted["pred_idx_min"]))

0.3036529680365297


In [73]:
print(accuracy(predicted["target"], predicted["pred_idx_che"]))

0.24733637747336376


In [57]:
# predicted.to_csv("train_detect_sent.csv", index=None)